# Decision Tree(CART)


In [1806]:

from csv import reader
from collections import Counter 

def load_csv(filename):
    file = open(filename, 'rt')
    lines = reader(file)
    dataset = [list(map( str,row)) for row in lines]
    return dataset


In [1807]:
def gini_index(n_in_class,n_yes,n_no):
    if(n_in_class>0):
        result = 1-pow((n_yes/n_in_class),2)-pow((n_no/n_in_class),2)
    #     print("888888:",pow((n_yes/n_in_class),2))
        if(result<0):
            result=0
    else:
        result=0
    return result


In [1808]:
def weighted_sum_gini (n_instance,prob):
    return(0)
    
    

In [1809]:
def decide_tree_root(data):
    dataset=data[1:]
    data_y = data[0:1]
    class_set=[[] for _ in range(len(dataset[0]))]
    for row in range(len(dataset)):
        for i in range(len(dataset[row])):
            class_set[i].append(dataset[row][i])
    sum_weight=weighted_sum_root(class_set)
    min_cost=sum_weight.index(min(sum_weight))
    return (data_y[0][min_cost],sum_weight[min_cost],min_cost)
        
    

In [1810]:
def weighted_sum_root(dataset):
#     print(dataset)
    class_set=([[] for _ in range(len(dataset)-1)])
    count_prob=[]
    prob_set=[]
    temp_prob=[]
    
    result=dataset[len(dataset)-1]
    for i in range(len(dataset)-1):
        prob_set.append( list( dict.fromkeys(dataset[i]) ))

        for j in range(len(dataset[i])):
            class_set[i].append([dataset[i][j],result[j]])
    count_prob_att=[]
    count_prob_target=[]
    for att in range(len(class_set)): #1 row
        for prob_att in range(len(prob_set[att])):
            p=0
            n=0
            target_prob = prob_set[att][prob_att]
            for index in range(len(class_set[att])): #1set
                x_sample = class_set[att][index][0]
                y_sample = class_set[att][index][1]
                if((x_sample == target_prob) and (y_sample =='Yes')):
                    p+=1
                elif((x_sample == target_prob) and (y_sample =='No')):
                    n+=1
            count_prob_target.append([target_prob,p,n])
        count_prob_att.append(count_prob_target)
        count_prob_target=[]
        
    weighted_sum_gini=[]
    count_weight=0
    for att in range(len(count_prob_att)):
        for i in range(len(count_prob_att[att])):
            event = count_prob_att[att][i][0]
            n_positive = event = count_prob_att[att][i][1]
            n_negative = event = count_prob_att[att][i][2]
            n_sample = n_positive+n_negative
            cost = gini_index(n_sample,n_positive,n_negative)
            count_weight+=((n_sample/len(dataset[0]))*cost)
        weighted_sum_gini.append(count_weight)
        count_weight=0
    return weighted_sum_gini

In [1811]:
def pair_root(data,index_root):
    
    attr = (data[0])
    attribute = attr[:]

    attribute.pop(index_root)
    attribute.pop(len(attribute)-1)

    dataset=data[1:]
    data_y = data[0:1]
    prob_set=[]
    class_set=[[] for _ in range(len(dataset[0])-1)]
    for row in range(len(dataset)):

        for i in range(len(dataset[row])-1):
            class_set[i].append(dataset[row][i])

    
    for att in range (len(class_set)):
        prob_set.append( list( dict.fromkeys(class_set[att]) ))
        
    n_positive=0
    n_negative=0
    result_set=[]
    for event_prob_root in range(len(prob_set[index_root])):
        path_set=[]
        for event_prob_pair in range(len(prob_set)):
            if(event_prob_pair != index_root):
                count_weight=0
                decision_set_weight=[]
                for index_child in range(len(prob_set[event_prob_pair])):
                    if(event_prob_pair != index_root):
                        for row in range(len(dataset)):
                            check_root = dataset[row][index_root] ==prob_set[index_root][event_prob_root]
                            check_pair = dataset[row][event_prob_pair] ==prob_set[event_prob_pair][index_child]
                            result = dataset[row][len(dataset[row])-1]
                            if(check_root and check_pair and result=="Yes"):
                                n_positive+=1
                            if(check_root and check_pair and result=="No"):
                                n_negative+=1
                        n_sample=n_positive+n_negative
                        cost = gini_index(n_sample,n_positive,n_negative)
                        n_positive=0
                        n_negative=0
                        count_weight+=((n_sample/len(dataset[0]))*cost)

                decision_set_weight.append(count_weight)
                path_set.append(decision_set_weight)
        result_set.append(path_set)
    return prob_set,result_set

In [1812]:

def calculate_path_layer(data,prob_set,result_set,index_root):
    att=data[0]
    attribute = att[:]
    attribute.pop(index_root)
    dataset=data[1:]
    nd_set=prob_set[:]
    nd_set.pop(index_root)
    path=[]
    for index in range(len(prob_set[index_root])):
        min_cost=result_set[index].index(min(result_set[index]))
        for i in range(len(nd_set[min_cost])):
            case = (nd_set[min_cost][i])
            for j in range(len(dataset)):
#                 print(dataset[j][index_root],prob_set[index_root][index]  )
                check_root = prob_set[index_root][index]  in dataset[j]
                check_pair = case in dataset[j]
                if(check_root and check_pair ):
#                     print("in",att[index_root],prob_set[index_root][index],attribute[min_cost],case,dataset[j][len(dataset[j])-1])
                    result_data =[att[index_root],prob_set[index_root][index],attribute[min_cost],case,dataset[j][len(dataset[j])-1]]
                    if(result_data not in (path)):
                        path.append(result_data)
    print(path)
    return(path)

In [1813]:

dataset=(load_csv("data/golf_playing_decisions.csv"))
attr = (dataset[0])
event_root,cost_root,index_root=decide_tree_root(dataset)
prob_set,weight_path_pair=pair_root(dataset,index_root)
path=calculate_path_layer(dataset,prob_set,weight_path_pair,index_root)


[['Outlook', 'Sunny', 'Humidity', 'High', 'No'], ['Outlook', 'Sunny', 'Humidity', 'Normal', 'Yes'], ['Outlook', 'Overcast', 'Temp.', 'Hot', 'Yes'], ['Outlook', 'Overcast', 'Temp.', 'Mild', 'Yes'], ['Outlook', 'Overcast', 'Temp.', 'Cool', 'Yes'], ['Outlook', 'Rain', 'Wind', 'Weak', 'Yes'], ['Outlook', 'Rain', 'Wind', 'Strong', 'No']]
